## Gymnasium environments

This Section shows how you can make and use the `gym` environments that interface with the simulator.

In [1]:
import os
from pathlib import Path
import torch
import mediapy

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

from pygpudrive.env.config import EnvConfig, RenderConfig, SceneConfig, SelectionDiscipline
from pygpudrive.env.env_torch import GPUDriveTorchEnv

### Settings

In [2]:
MAX_NUM_OBJECTS = 128  # Maximum number of objects in the scene we control
NUM_WORLDS = 4  # Number of parallel environments
K_UNIQUE_SCENES = 3 # Number of unique scenes

# Set the path to where you want to save the videos
VIDEO_PATH = "./videos"

FPS = 4  # Video frames per second

device = 'cpu' # for explanation purposes in notebook #torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Initializing environments

- We provide both a torch and jax gymnasium interface with the simulator. Most functionality is specified in the `GPUDriveGymEnv` class in the `base_env`, `torch_env` and `jax_env` both inherit from the `GPUDriveGymEnv`, the only difference between these is that one exports torch tensors and the other jax arrays.
- All data settings are defined in the `SceneConfig` dataclass.
- All environment settings are defined in the `EnvConfig` dataclass. 
- All rendering configs are defined in the `RenderConfig` dataclass.


In [3]:
scene_config = SceneConfig(
    path="data/processed/examples", 
    num_scenes=NUM_WORLDS,
    discipline=SelectionDiscipline.K_UNIQUE_N,
    k_unique_scenes=K_UNIQUE_SCENES,
)

In [4]:
env_config = EnvConfig(
    steer_actions = torch.round(
        torch.linspace(-1.0, 1.0, 3), decimals=3),
    accel_actions = torch.round(
        torch.linspace(-3, 3, 3), decimals=3
    )
)

In [9]:
render_config = RenderConfig(
    resolution=(800, 800), # Quality of the rendered images
    draw_obj_idx=True, # Draw object index on the objects
)

---

> 💡 **For more info about the environment configurations, see `pygpudrive/env/README.md`**

---

In [10]:
# MAKE ENV
env = GPUDriveTorchEnv(
    config=env_config,
    scene_config=scene_config,
    render_config=render_config,
    max_cont_agents=MAX_NUM_OBJECTS, # Maximum number of agents to control per scenario
    device=device,
)


--- Ratio unique scenes / number of worls = 3 / 4 ---



### Run an episode with `N` parallel environments

A single rollout (one episode) is implemented as follows:
- We step through N worlds simultaneously.
- Use the `world_render_idx` argument in `render(.)` to select which world to render.

In [11]:
obs = env.reset()
frames = {f"env_{i}": [] for i in range(NUM_WORLDS)}

for _ in range(env_config.episode_len):
    # SELECT ACTIONS
    rand_action = torch.Tensor(
        [[env.action_space.sample() for _ in range(MAX_NUM_OBJECTS * NUM_WORLDS)]]
    ).reshape(NUM_WORLDS, MAX_NUM_OBJECTS)

    # STEP
    env.step_dynamics(rand_action)

    obs = env.get_obs()
    reward = env.get_rewards()
    done = env.get_dones()

    # RENDER
    for i in range(NUM_WORLDS):
        frames[f"env_{i}"].append(env.render(i))

### Display videos (of agents taking random actions)

In [13]:
# Show videos
mediapy.show_videos(frames, fps=30, width=600, height=400, columns=2)

env_0 This browser does not support the video tag.,env_1 This browser does not support the video tag.
env_2 This browser does not support the video tag.,env_3 This browser does not support the video tag.
